Copyright 2024 Lehi Gracia

### Markets

In [30]:

import warnings
warnings.filterwarnings('ignore')

from yfinance_utils import list_utils, file_utils, constants
import yfinance 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from finta import TA

COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5, width=600, height=400)

data = yfinance.download(symbols, period='1y')
gains = {}
for symbol in symbols:
    df1 = pd.DataFrame()
    df1 = data.loc[:,(slice(None),symbol)]
    df1.columns = COLUMNS
    df1['sma'] = TA.SMA(df1, period=50)
    df1 = df1[50:]
    df1.reset_index(inplace=True)

    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(y=df1['sma'], x=df1['Date'], name=f'sma'),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_2 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()

print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:4}:{gains[names[symbol]]:6}: {names[symbol]}')



[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:
QQQ :  1.02: NASDAQ 100
SPY :  -1.8: SNP 500
DIA : -4.75: DOW JONES


### Sectors

In [31]:

from yfinance_utils import list_utils

symbols = list_utils.get_sectors()
names = list_utils.sectors
COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']


top = list_utils.get_top_performers(symbols)
_list = [i[0] for i in top]

data = yfinance.download(_list, period='1y')
for symbol in _list:
    fig2=make_subplots(specs=[[{"secondary_y": True}]])
    fig2.update_layout(title="TOP 3 SECTORS", title_x=0.5, width=600, height=400)
    _d2 = pd.DataFrame()
    _d2 = data.loc[:,(slice(None),symbol)]
    _d2.columns = COLUMNS
    _d2['sma'] = TA.SMA(_d2, period=50)
    _d2.reset_index(inplace=True)
    fig2.add_trace(
        go.Scatter(y=_d2['Close'], x=_d2['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig2.add_trace(
        go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
        secondary_y=False
    )
    fig2.show()


print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol, value in top:
    print(f'{symbol:5}:{names[symbol]:8}: {value}')


[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:
XLY  :CONSUMER DISCRETIONARY: 0.72
XLC  :COMMUNICATION SERVICES: 0.0
XLK  :TECHNOLOGY: -0.37


###  Getting 5 best performing stocks in each of top 3 sectors

In [32]:

from yfinance_utils import list_utils
_sym = {}
for symbol, value in top:
    _l = []
    for s in list_utils.SECTORS[symbol].keys():
        _l.append(s)
    _tp = list_utils.get_top_performers(_l, size=5)
    _l = [i[0] for i in _tp]
    _sym[symbol] = _l
    
for _s in _sym:
    data = yfinance.download(_sym[_s], period='1y')
    for symbol in _sym[_s]:
        _f=make_subplots(specs=[[{"secondary_y": True}]])
        _f.update_layout(title=f"TOP from {names[_s]}", title_x=0.5, width=600, height=400)
        _d2 = pd.DataFrame()
        _d2 = data.loc[:,(slice(None),symbol)]
        _d2.columns = COLUMNS
        _d2['sma'] = TA.SMA(_d2, period=50)
        _d2.reset_index(inplace=True)
        _f.add_trace(
            go.Scatter(y=_d2['Close'], x=_d2['Date'], name=symbol),
            secondary_y=False
        )
        _f.add_trace(
            go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
            secondary_y=False
        )
        _f.show()



[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  70 of 70 completed

1 Failed download:
['XAKH25']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


### Top Analysts Ratings

In [33]:

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_colwidth', None)


client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.market

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':today, 'TOTAL':{"$gt":40}, 'script':'daily_analysts_ratings'}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
print(data)


          DATE  TICK  UP  DOWN  HOLD    AVG  TOTAL
0   2025-01-08  AMZN  63     1     3  94.03     67
1   2025-01-08   WMT  38     1     3  90.48     42
2   2025-01-08  DDOG  40     0     6  86.96     46
3   2025-01-08  MSFT  52     0     5  91.23     57
4   2025-01-08  META  58     2     8  85.29     68
5   2025-01-08  NVDA  59     0     4  93.65     63
6   2025-01-08  AVGO  38     0     5  88.37     43
7   2025-01-08  AMZN  63     1     3  94.03     67
8   2025-01-08   WMT  38     1     3  90.48     42
9   2025-01-08  DDOG  40     0     6  86.96     46
10  2025-01-08  MSFT  52     0     5  91.23     57
11  2025-01-08  META  58     2     8  85.29     68
12  2025-01-08  NVDA  59     0     4  93.65     63


### Gap up close high

In [34]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_gap_up_close_high']}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
print(data)

          DATE  TICK    HIGH   CLOSE    OPEN  CLOSE -1
0   2025-01-08   CAH  122.72  122.65  122.53    120.67
1   2025-01-08   DGX  154.28  154.28  154.03    153.28
2   2025-01-08   DHR  239.39  238.61  238.09    238.00
3   2025-01-08   BSX   97.13   96.68   95.95     91.70
4   2025-01-08  QRVO   73.33   73.13   73.08     72.68
5   2025-01-08     A  137.68  137.68  137.00    136.83
6   2025-01-08   MKC   74.76   74.54   74.52     74.11
7   2025-01-08   CHD  102.97  102.63  102.57    102.24
8   2025-01-08   TRV  243.96  243.51  242.77    241.25
9   2025-01-08   BMY   57.05   56.98   56.81     56.68
10  2025-01-08    LH  236.03  235.10  234.31    232.51


### Price drops and jumps

In [35]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_jump_price_down','daily_jump_price_up']}}, 
                 {'_id':0, 'script':0, 'timestamp':0, 'VOLUME':0  }
                 )

data = pd.DataFrame(res).round(2)
print(data)


          DATE  TICK   CLOSE  % PRICE JUMP  % VOLUME -1  % VOLUME -2
0   2025-01-08  MRNA   47.50          8.20       -41.43       163.39
1   2025-01-08   BSX   96.68          5.43       123.15        -3.40
2   2025-01-08  EBAY   67.90          6.69       461.73       -29.95
3   2025-01-08   SYM   25.86         -5.96       -17.93        -6.29
4   2025-01-08  PLTR   68.14         -9.39        -2.87       -13.15
5   2025-01-08  QBTS    6.58        -36.97       171.51       -29.42
6   2025-01-08  ULTA  425.00         -5.56       -58.46       166.57
7   2025-01-08  ACHR   10.68         -6.89       -23.09       -17.22
8   2025-01-08  WULF    5.91         -9.22         9.96       -15.71
9   2025-01-08   SMR   20.53        -12.97       -21.51       -25.84
10  2025-01-08  IONQ   38.75        -24.43       358.15        16.27
11  2025-01-08  LUNR   19.01        -12.76        -7.79        -4.03
12  2025-01-08  MSTR  335.24         -8.52       -27.53        -7.38
13  2025-01-08  SOUN   16.55      